https://github.com/kokoff/kernighan-lin

https://github.com/kokoff/kernighan-lin

file:///home/berend/Downloads/Handbook%20of%20Approximation%20Algorithms%20and%20Metaheuristics%20[Gonzalez%202007-01-05].pdf

local search with variable depth

out of approaches choose the one that makes you happy

# Initialize problem

In [1]:
s = """c FILE: myciel3.col
c SOURCE: Michael Trick (trick@cmu.edu)
c DESCRIPTION: Graph based on Mycielski transformation. 
c              Triangle free (clique number 2) but increasing
c              coloring number
p edge 11 20
e 1 2
e 1 4
e 1 7
e 1 9
e 2 3
e 2 6
e 2 8
e 3 5
e 3 7
e 3 10
e 4 5
e 4 6
e 4 10
e 5 8
e 5 9
e 6 11
e 7 11
e 8 11
e 9 11
e 10 11"""

In [2]:
import numpy as np
import math
import sys
import fileinput

In [3]:
# s = """c FILE: myciel3.col
# c SOURCE: Michael Trick (trick@cmu.edu)
# c DESCRIPTION: Graph based on Mycielski transformation. 
# c              Triangle free (clique number 2) but increasing
# c              coloring number
# p edge 8 13
# e 1 2
# e 1 5
# e 1 6
# e 2 5
# e 2 6
# e 3 4
# e 3 6
# e 3 7
# e 3 8
# e 4 7
# e 4 8
# e 5 6
# e 7 8"""

In [4]:
# Read data from string
def read_graph(s):
    for line in s.splitlines():
        if line.startswith('c'):
            continue
        elif line.startswith('p'):
            line = line.split()
            vertices_number, edges_number = int(line[-2]), int(line[-1])
            edges = []
            datad = [0] * vertices_number
        elif line.startswith('e'):
            line = line.split()
            v1, v2 = int(line[-2]), int(line[-1])
            edges.append([int(line[-2]), int(line[-1])])
    edges = np.asarray(edges)
    return vertices_number, edges_number, edges

In [5]:
def connect_matrix(vertices_number, edges_number, edges):
    CM1 = np.zeros(shape=(vertices_number,vertices_number))
    for i in range(0, edges_number):
    #     print('---------------------------')
    #     print('edge', i)
    #     print(edges[i])
    #     print('fill matrix')
        n1 = edges[i][0] - 1
        n2 = edges[i][1] - 1
    #     print(n1, n2)
    #     print(A)
        CM1[n1,n2] = 1
        CM2 = np.rot90(np.fliplr(CM1))
               
    return (CM1 + CM2).astype(int)

In [6]:
def partition_init(vertices_number, edges_number, edges):
    # Split vertices in a random partition
    partit_A_init = np.linspace(
        1, 
        math.ceil(vertices_number/2),
        math.ceil(vertices_number/2), 
        endpoint=True)

    partit_B_init = np.linspace(
        math.ceil(vertices_number/2) + 1,
        vertices_number, 
        vertices_number+1-(math.ceil(vertices_number/2) + 1),
        endpoint=True)

#     print(vertices_number,edges_number)
#     print(edges)
#     print(partit_A_init)
#     print(partit_B_init)

    return partit_A_init.astype(int), partit_B_init.astype(int)

In [7]:
def D_vector(A, B, CM):
    ext_costA = np.zeros((vertices_number, 1))
    int_costA = np.zeros((vertices_number, 1))
    for i in A:.splitlines()
#         print(' ')
#         print('row', i )
        for j in B:
#             print('(i,j) = ', i , j )
#             print('CM(i,j)= ', CM[i-1,j-1])
            ext_costA[i-1] += CM[i-1,j-1]
#         print(ext_costA[i-1])

    for i in A:
#         print(' ')
#         print('row', i )
        for j in A:
            if i != j:
#                 print('(i,j) = ', i , j )
#                 print('CM(i,j)= ', CM[i-1,j-1])
                int_costA[i-1] += CM[i-1,j-1] 


    ext_costB = np.zeros((vertices_number, 1))
    int_costB = np.zeros((vertices_number, 1))
    for i in B:
#         print(' ')
#         print('row', i )
        for j in A:
#             print('(i,j) = ', i , j )
#             print('CM(i,j)= ', CM[i-1,j-1])
            ext_costB[i-1] += CM[i-1,j-1]  

    for i in B:
#         print(' ')
#         print('row', i )
        for j in B:
#             print(j < 1)
            if j != i:
#                 print('(i,j) = ', i , j )
#                 print('CM(i,j)= ', CM[i-1,j-1])
                int_costB[i-1] += CM[i-1,j-1] 

    ext_cost = ext_costA + ext_costB
    int_cost = int_costA + int_costB

    D =  ext_cost - int_cost
       
    return D

In [8]:
def max_gain(A, B, D, CM):
    maxGain = -sys.maxsize - 1
    n1 = None
    n2 = None
    for i in A:
        for j in B:
            gain = D[i-1] + D[j-1] - 2 * CM[i-1,j-1]
            gain = gain[0]
            if gain > maxGain:
                maxGain = gain
                n1 = i
                n2 = j
    return n1, n2, maxGain

In [9]:
def getMaxGainAndIndex(gains):
    maxGain = -sys.maxsize - 1
    index = 0
    sum = 0

    for i in gains:
        sum += i
        if sum > maxGain:
            maxGain = sum
            index = gains.index(i)
    return maxGain, index

In [10]:
def updateD(CM, A, B, D, n1, n2):
    for i in A:
        D[i-1] = D[i-1] + 2*CM[i-1,n1-1] - 2*CM[i-1,n2-1]
    for i in B:
        D[i-1] = D[i-1] + 2*CM[i-1,n2-1] - 2*CM[i-1,n1-1]
    return D

In [11]:
def switch(A, B, CM):
    D = D_vector(A, B, CM)
    gains = []
    X = []
    Y = []


    for i in range( math.floor(len(CM) / 2) ):
#         print(' ')
#         print('-------------')
#         print('iteration', i)

        n1, n2, gain = max_gain(A, B, D, CM)  

#         print('nodes to switch', n1, n2)
        
#         print(A)
#         print(B)

        A = A[A != n1]
        B = B[B != n2]

        gains.append(gain)

        X.append(n1)
        Y.append(n2)

        D = updateD(CM, A, B, D, n1, n2)

    maxGain, k = getMaxGainAndIndex(gains)
    
    #print('MAXGAIN', maxGain)

    if maxGain > 0:
        A = np.array(Y[:k + 1] + X[k + 1:])
        B = np.array(X[:k + 1] + Y[k + 1:])
        return A, B, False
    else:
        A = [i for i in X]
        B = [i for i in Y]
        return A, B, True

# Run

In [13]:
# vertices_number, edges_number, edges = read_graph(fileinput.input())

vertices_number, edges_number, edges = read_graph(s)
CM = connect_matrix(vertices_number, edges_number, edges)
A, B = partition_init(vertices_number, edges_number, edges)

done = False
while not done:
    A, B, done = switch(A, B, CM)

# print(A)
# print(B)

print(*A)

[10, 3, 6, 4, 2]
[8, 1, 9, 7, 11]
10 3 6 4 2
